# Исследование надежности заемщиков

Это первая часть проекта, она будет проверена автоматически. Вторую часть проверит ревьюер. 

## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [3]:
import pandas as pd   # импортируйте библиотеку pandas

In [4]:
data = pd.read_csv('/datasets/data.csv')  # прочитайте csv-файл

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [5]:
print(data.head(20)) # ваш код здесь

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [6]:
data.info()   # ваш код здесь

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [7]:
print(data.isna().sum())   # ваш код здесь

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [8]:
for job in data['income_type'].unique():
    data.loc[(data['income_type'] == job) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == job), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [9]:
data['days_employed'] = abs(data['days_employed'])

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [10]:
data.groupby('income_type')['days_employed'].median()
    # ваш код здесь

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [11]:
data['children'].unique()   # ваш код здесь

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [12]:
data = data[(data['children']>-1) & (data['children']!=20)]


**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [13]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [14]:
for job in data['income_type'].unique():
    data.loc[(data['income_type'] == job) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == job), 'days_employed'].median()                                     

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [15]:
data.value_counts()

children  days_employed  dob_years  education            education_id  family_status     family_status_id  gender  income_type  debt  total_income   purpose                               
0         365286.622650  62         среднее              1             женат / замужем   0                 F       пенсионер    0     118514.486412  ремонт жилью                              3
          1572.328285    47         среднее              1             женат / замужем   0                 F       сотрудник    0     142594.396847  ремонт жилью                              3
1         1572.328285    44         среднее              1             женат / замужем   0                 F       сотрудник    0     142594.396847  сделка с подержанным автомобилем          2
2         1572.328285    39         среднее              1             гражданский брак  1                 F       сотрудник    0     142594.396847  сыграть свадьбу                           2
1         1555.947387    31         сред

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [16]:
data['total_income'] = data['total_income'].astype(int)# ваш код здесь

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [17]:
data['education']=data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [18]:
data.duplicated().sum()

71

In [19]:
data=data.drop_duplicates().reset_index(drop=True)

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [20]:
def categorize_income(total_income):
    if total_income >=0 and total_income <=30000:
        return 'E'
    if total_income >= 30001 and total_income <= 50000:
        return 'D'
    if total_income >=50001 and total_income <= 200000:
        return 'C'
    if total_income >= 200001 and total_income <= 1000000:
        return 'B'
    return 'A'
    


In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income) # примените функцию методом apply()

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [22]:
data['purpose'].unique()# ваш код здесь

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [30]:
def categorize_purpose(purpose):
    
    if  'авто' in purpose:
        return 'операции с автомобилем'
    if  'недвиж'    in purpose:
        return 'операции с недвижимостью'
    
    if  'свад' in purpose:
        return 'проведение свадьбы'
    if  'образ' in purpose:
        return 'получение образования'

In [31]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
print(data['purpose'].head(15))
print()
print(data['purpose_category'].head(15))# примените функцию методом apply()

0                         покупка жилья
1               приобретение автомобиля
2                         покупка жилья
3            дополнительное образование
4                       сыграть свадьбу
5                         покупка жилья
6                     операции с жильем
7                           образование
8                 на проведение свадьбы
9               покупка жилья для семьи
10                 покупка недвижимости
11    покупка коммерческой недвижимости
12                      сыграть свадьбу
13              приобретение автомобиля
14           покупка жилой недвижимости
Name: purpose, dtype: object

0                         None
1       операции с автомобилем
2                         None
3        получение образования
4           проведение свадьбы
5                         None
6                         None
7        получение образования
8           проведение свадьбы
9                         None
10    операции с недвижимостью
11    операции с недвижимость